In [2]:
import pandas as pd
import numpy as np
import os
import pickle
import gcsfs
from datetime import datetime
from functools import reduce
from google.cloud import bigquery
from tqdm import tqdm


# Query demographics data for ADRD patients

In [3]:

my_bucket = os.getenv("WORKSPACE_BUCKET")


In [ ]:
import pandas
import os

def query_cohort(querytype):


    def demographics_query_uf_case():
        
        # person_id, gender_concept_id, gender_concept_name, 
        # birth_datetime, -
        # race_concept_id, race_concept_name, 
        # ethnicity_concept_id, ethnicity_concept_name, 
        # sex_at_birth_concept_id, sex_at_birth_concept_name
        
        uf_case_sql = """
            SELECT
                person.person_id,
                person.gender_concept_id,
                p_gender_concept.concept_name as gender,
                person.birth_datetime as date_of_birth,
                person.race_concept_id,
                p_race_concept.concept_name as race,
                person.ethnicity_concept_id,
                p_ethnicity_concept.concept_name as ethnicity,
                person.sex_at_birth_concept_id,
                p_sex_at_birth_concept.concept_name as sex_at_birth 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
                    ON person.gender_concept_id = p_gender_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
                    ON person.race_concept_id = p_race_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
                    ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
                    ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
            WHERE
                person.PERSON_ID IN (
                    SELECT
                        distinct person_id  
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                    WHERE
                        cb_search_person.person_id IN (
                            SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        concept_id IN(
                                            SELECT
                                                DISTINCT c.concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                            JOIN
                                                (
                                                    SELECT
                                                        CAST(cr.id as string) AS id       
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                                    WHERE
                                                        concept_id IN (44831122, 35207495, 44823009, 45547675, 44832265, 35207362, 35207832, 44826539, 44820749, 35207116, 44824106, 35211390, 45562080, 35207397, 44837000, 35207508, 44820073, 35207121, 35207829, 44827645, 45533468, 44819535, 45576890, 45591073, 44823010, 45547690, 44820708, 45595932, 35207835, 44835772, 35207359, 44827641, 1568295, 35207497, 44833397, 45600684, 35207830, 44832388, 35207125, 45547730, 44833434, 44831078, 44829914, 35207834, 35207358, 45600775, 44821814, 35207360, 45553736, 35207356, 35207117, 45566776, 45890911, 35207833, 35207357, 1568090, 45552458, 1568089, 35207361, 44832219, 44821811, 35207496, 35207837, 35207511, 44826489, 45581787, 44826538, 44825327, 35207115, 44829915, 35207118, 44833435, 44821813, 44826537)       
                                                        AND full_text LIKE '%_rank1]%'      
                                                ) a 
                                                    ON (
                                                        c.path LIKE CONCAT('%.',
                                                    a.id,
                                                    '.%') 
                                                    OR c.path LIKE CONCAT('%.',
                                                    a.id) 
                                                    OR c.path LIKE CONCAT(a.id,
                                                    '.%') 
                                                    OR c.path = a.id) 
                                                WHERE
                                                    is_standard = 0 
                                                    AND is_selectable = 1
                                                ) 
                                                AND is_standard = 0 
                                        )
                                    ) criteria 
                                ) 
                                AND cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        age_at_cdr BETWEEN 40 AND 120 
                                        AND NOT EXISTS (
                                            SELECT
                                                'x'      
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                                            WHERE
                                                d.person_id = p.person_id 
                                        ) 
                                    ) 
                                    AND cb_search_person.person_id IN (
                                        SELECT
                                            person_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                        WHERE
                                            has_ehr_data = 1 
                                    ) 
                                )"""

        #         uf_case = pandas.read_gbq(
        #             uf_case_sql,
        #             dialect="standard",
        #             use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        #             progress_bar_type="tqdm_notebook")

        #         uf_case.head(5)
        return uf_case_sql

    
    def demographics_query_ucsf_control():
        
        ucsf_control_sql = """
            SELECT
                person.person_id,
                person.gender_concept_id,
                p_gender_concept.concept_name as gender,
                person.birth_datetime as date_of_birth,
                person.race_concept_id,
                p_race_concept.concept_name as race,
                person.ethnicity_concept_id,
                p_ethnicity_concept.concept_name as ethnicity,
                person.sex_at_birth_concept_id,
                p_sex_at_birth_concept.concept_name as sex_at_birth 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
                    ON person.gender_concept_id = p_gender_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
                    ON person.race_concept_id = p_race_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
                    ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
                    ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
            WHERE
                person.PERSON_ID IN (
                    SELECT
                        distinct person_id  
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                    WHERE
                        cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                has_ehr_data = 1 
                        ) 
                        AND cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                age_at_cdr BETWEEN 40 AND 120 
                                AND NOT EXISTS (
                                    SELECT
                                        'x'      
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                                    WHERE
                                        d.person_id = p.person_id 
                                ) 
                            ) 
                            AND cb_search_person.person_id NOT IN (
                                SELECT
                                    criteria.person_id 
                                FROM
                                    (SELECT
                                        DISTINCT person_id,
                                        entry_date,
                                        concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                    WHERE
                                        person_id IN (
                                            SELECT
                                                person_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                concept_id IN(
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            SELECT
                                                                CAST(cr.id as string) AS id       
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                                            WHERE
                                                                concept_id IN (4182210)       
                                                                AND full_text LIKE '%_rank1]%'      
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            is_standard = 1 
                                                            AND is_selectable = 1
                                                        ) 
                                                        AND is_standard = 1 
                                                    UNION
                                                    DISTINCT SELECT
                                                        person_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                    WHERE
                                                        concept_id IN(
                                                            SELECT
                                                                DISTINCT c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    SELECT
                                                                        CAST(cr.id as string) AS id       
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                                                    WHERE
                                                                        concept_id IN (1568280, 1568294, 1568297, 35207365, 35207358, 35207359, 1568284, 1568279, 920130, 1568299, 35207356, 1568298, 35207357, 1568287, 1568282, 1568286, 35207314, 1568289, 35207329)       
                                                                        AND full_text LIKE '%_rank1]%'      
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    is_standard = 0 
                                                                    AND is_selectable = 1
                                                                ) 
                                                                AND is_standard = 0 
                                                        )
                                                    ) criteria 
                                                UNION
                                                DISTINCT SELECT
                                                    criteria.person_id 
                                                FROM
                                                    (SELECT
                                                        DISTINCT person_id,
                                                        entry_date,
                                                        concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                    WHERE
                                                        (
                                                            concept_id IN(
                                                                SELECT
                                                                    DISTINCT ca.descendant_id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria_ancestor` ca 
                                                                JOIN
                                                                    (
                                                                        SELECT
                                                                            DISTINCT c.concept_id       
                                                                        FROM
                                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c       
                                                                        JOIN
                                                                            (
                                                                                SELECT
                                                                                    CAST(cr.id as string) AS id             
                                                                                FROM
                                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr             
                                                                                WHERE
                                                                                    concept_id IN (21604792)             
                                                                                    AND full_text LIKE '%_rank1]%'       
                                                                            ) a 
                                                                                ON (
                                                                                    c.path LIKE CONCAT('%.',
                                                                                a.id,
                                                                                '.%') 
                                                                                OR c.path LIKE CONCAT('%.',
                                                                                a.id) 
                                                                                OR c.path LIKE CONCAT(a.id,
                                                                                '.%') 
                                                                                OR c.path = a.id) 
                                                                            WHERE
                                                                                is_standard = 1 
                                                                                AND is_selectable = 1
                                                                            ) b 
                                                                                ON (
                                                                                    ca.ancestor_id = b.concept_id
                                                                                )
                                                                        ) 
                                                                        AND is_standard = 1
                                                                    )
                                                            ) criteria 
                                                        ) 
                                                    )"""

        #         ucsf_control = pandas.read_gbq(
        #             ucsf_control_sql,
        #             dialect="standard",
        #             use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        #             progress_bar_type="tqdm_notebook")

        #         ucsf_control.head(5)
        return ucsf_control_sql



    def demographics_query_uf_case_large():
        dataset_45923547_person_sql = """
            SELECT
                person.person_id,
                person.gender_concept_id,
                p_gender_concept.concept_name as gender,
                person.birth_datetime as date_of_birth,
                person.race_concept_id,
                p_race_concept.concept_name as race,
                person.ethnicity_concept_id,
                p_ethnicity_concept.concept_name as ethnicity,
                person.sex_at_birth_concept_id,
                p_sex_at_birth_concept.concept_name as sex_at_birth 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
                    ON person.gender_concept_id = p_gender_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
                    ON person.race_concept_id = p_race_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
                    ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
                    ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
            WHERE
                person.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (44831122, 35207495, 44823009, 45547675, 44832265, 35207362, 35207832, 44826539, 44820749, 35207116, 44824106, 35211390, 45562080, 35207397, 44837000, 35207508, 44820073, 35207121, 35207829, 44827645, 45533468, 44819535, 45576890, 45591073, 44823010, 45547690, 44820708, 45595932, 35207835, 44835772, 35207359, 44827641, 1568295, 35207497, 44833397, 45600684, 35207830, 44832388, 35207125, 45547730, 44833434, 44831078, 44829914, 35207834, 35207358, 45600775, 44821814, 35207360, 45553736, 35207356, 35207117, 45566776, 45890911, 35207833, 35207357, 1568090, 45552458, 1568089, 35207361, 44832219, 44821811, 35207496, 35207837, 35207511, 44826489, 45581787, 44826538, 44825327, 35207115, 44829915, 35207118, 44833435, 44821813, 44826537)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) )"""

        return dataset_45923547_person_sql


    def demographics_query_uf_control_large():
        dataset_79779476_person_sql = """
            SELECT
                person.person_id,
                person.gender_concept_id,
                p_gender_concept.concept_name as gender,
                person.birth_datetime as date_of_birth,
                person.race_concept_id,
                p_race_concept.concept_name as race,
                person.ethnicity_concept_id,
                p_ethnicity_concept.concept_name as ethnicity,
                person.sex_at_birth_concept_id,
                p_sex_at_birth_concept.concept_name as sex_at_birth 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
                    ON person.gender_concept_id = p_gender_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
                    ON person.race_concept_id = p_race_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
                    ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
                    ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
            WHERE
                person.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) )"""


        return dataset_79779476_person_sql
        


    # for case
    if querytype == 'uf_case_large':
        query = demographics_query_uf_case_large()
    
    # for control
    if querytype == 'uf_control_large':
        query = demographics_query_uf_control_large()
        
    return pandas.read_gbq(
                query,
                dialect="standard",
                use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
                progress_bar_type="tqdm_notebook")





## query cohort

In [ ]:
ADRD_cases = query_cohort('uf_case_large')
ADRD_controls = query_cohort('uf_control_large')

ADRD_patient_group_ids = ADRD_cases['person_id'].unique().tolist()
ADRD_control_group_ids = ADRD_controls['person_id'].unique().tolist()

# Functions: query dates-related information

In [58]:
def query_data_dates(table_name, subject_ids, date_name):
    query = f"""
    SELECT DISTINCT person_id, {date_name} as record_date
    FROM {os.environ["WORKSPACE_CDR"]}.{table_name}
    WHERE person_id IN ({', '.join(map(str, subject_ids))})
    """
    data = pd.read_gbq(query, 
                     dialect="standard",
                     use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
                     progress_bar_type="tqdm_notebook")
    return data

## query other available EHR data for cases

In [ ]:
visit_data = query_data_dates("visit_occurrence", ADRD_patient_group_ids, "visit_start_date")
condition_data = query_data_dates("condition_occurrence", ADRD_patient_group_ids, "condition_start_date")
drug_exposure_data = query_data_dates("drug_exposure", ADRD_patient_group_ids, "drug_exposure_start_date")
measurement_data = query_data_dates("measurement", ADRD_patient_group_ids, "measurement_date")
procedure_data = query_data_dates("procedure_occurrence", ADRD_patient_group_ids, "procedure_date")
data_list_patients = [visit_data, condition_data, drug_exposure_data, measurement_data, procedure_data]

In [61]:
# record all unique encounter dates for each patient
date_information_patients = pd.concat(data_list_patients, ignore_index=True).sort_values(["person_id", "record_date"], ascending=False).drop_duplicates(subset=["person_id", "record_date"], keep="first").reset_index(drop=True)

In [62]:
def get_record_span(df):
    df["record_date"] = pd.to_datetime(df["record_date"])
    min_date = df["record_date"].dt.date.min()
    max_date = df["record_date"].dt.date.max()
    return ((max_date - min_date).days)/365
medical_history_span = date_information_patients.groupby("person_id").apply(lambda x: get_record_span(x)).reset_index(name="record span")


## select patients have at least 1 year of medical history, if needed

In [63]:
patients_not_enough_records = medical_history_span[medical_history_span["record span"] < 1]["person_id"]
ADRD_patient_group_ids = list(set(ADRD_patient_group_ids) - set(patients_not_enough_records))


## query other available EHR data for controls

In [67]:
# better in batch
batch_size = 50000  # Adjust this size as needed
batches = [ADRD_control_group_ids[i:i + batch_size] for i in range(0, len(ADRD_control_group_ids), batch_size)]

In [ ]:
#execute queries for each batch and concatenate the results
visit_control_data = pd.concat([query_data_dates("visit_occurrence", batch, "visit_start_date") for batch in batches], ignore_index=True)
condition_control_data = pd.concat([query_data_dates("condition_occurrence", batch, "condition_start_date") for batch in batches], ignore_index=True)
drug_exposure_control_data = pd.concat([query_data_dates("drug_exposure", batch, "drug_exposure_start_date") for batch in batches], ignore_index=True)
measurement_control_data = pd.concat([query_data_dates("measurement", batch, "measurement_date") for batch in batches], ignore_index=True)
procedure_control_data = pd.concat([query_data_dates("procedure_occurrence", batch, "procedure_date") for batch in batches], ignore_index=True)
data_list_control = [visit_control_data, condition_control_data, drug_exposure_control_data, measurement_control_data, procedure_control_data]


In [ ]:
#pickle the data frames for the control samples
for data, file_name in zip(data_list_control, ["visit_control", "condition_control", "drug_control", "measurement_control", "procedure_control"]):
    data.to_pickle(f"{file_name}.pkl")

## Filter the controls to have at least one year ehr

In [80]:
# data_list_control = []
# for file_name in ["visit_control", "condition_control", "drug_control", "measurement_control", "procedure_control"]:
#     data_unpacked = pd.read_pickle(f"{file_name}.pkl")
#     data_list_control.append(data_unpacked)
    
date_information_controls = pd.concat(data_list_control, ignore_index=True).sort_values(["person_id", "record_date"], ascending=False).drop_duplicates(subset=["person_id", "record_date"], keep="first").reset_index(drop=True)


In [81]:
medical_history_span_controls = date_information_controls.groupby("person_id").apply(lambda x: get_record_span(x)).reset_index(name="record span")
# medical_history_span_controls.to_pickle("medical_history_span_controls.pkl")


In [83]:
#filter out controls that do not have at least one year of medical records in the database
controls_not_enough_records = medical_history_span_controls[medical_history_span_controls["record span"] < 1]["person_id"]
ADRD_patient_group_ids = list(set(ADRD_patient_group_ids) - set(controls_not_enough_records))
